In [5]:
import argparse
import json
import logging
import shutil

import openrouteservice as ors
import pandas as pd

from advanced_geoscripting.scripts.utils import load_config
from advanced_geoscripting.scripts.filepaths import FilePaths

In [6]:
df = pd.read_csv("../run_pairs.csv")
df

,Unnamed: 0,lon,lat,lon2,lat2,distance
0,0,8.681673,49.436255,8.664183,49.381690,6.194010
1,1,8.626623,49.429120,8.638298,49.377357,5.813829
2,2,8.657504,49.418083,8.752665,49.396174,7.298887
3,3,8.747789,49.408750,8.605112,49.424165,10.455903
4,4,8.688200,49.369890,8.745959,49.424481,7.365509
...,...,...,...,...,...,...
85,95,8.637173,49.410095,8.690562,49.417784,3.953267
86,96,8.703606,49.366836,8.702268,49.416198,5.486261
87,97,8.714224,49.411343,8.641492,49.406389,5.287611
88,98,8.698501,49.403413,8.755992,49.424489,4.770830


In [9]:
df['lon'][2]

np.float64(8.657503792414388)

In [11]:
headers = {
        "headers": {
            "Accept": "application/json, application/geo+json, "
            "application/gpx+xml, img/png; charset=utf-8",
            "Content-Type": "application/json; charset=utf-8",
        }
    }

ors_url = "https://heal.openrouteservice.org/api-iso/ors/"

client = ors.Client(base_url=ors_url)

start_coordinate = [df['lon'][2], df['lat'][1]]
destination_coordinate = [df['lon2'][2], df['lat2'][1]]

profile = "foot-walking"


parameters = {
        "coordinates": [start_coordinate, destination_coordinate],
        "instructions": "false",
        "preference": "recommended",
        "extra_info": ["csv"],
        "elevation": "true",
        "continue_straight": "true",
        "options": {
            "avoid_features": ["ferries"],
            "profile_params": {"weightings": {"csv_factor": 1, "csv_column": "noon"}},
        },
    }

ors_response = client.request(
        url=f"v2/directions/{profile}/geojson",
        post_json=parameters,
        requests_kwargs=headers,
    )

with open("../data/run_v1/01_raw/01_routes/route_1.geojson", "w") as f:
    json.dump(ors_response, f)